In [ ]:
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
source_engine = create_engine('sqlite:///:memory:')
dest_engine = create_engine('sqlite:///:memory:')

In [ ]:
customers_data = {
    'id': [101, 102, 103, 104],
    'name': ['Alice', 'Bob', 'Charlie', 'David'],
    'gender' : ['P', 'L', 'L', 'L']
}
items_data = {
    'id': [201, 202, 203, 204, 205],
    'name': ['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Webcam'],
    'price': [1200.00, 25.00, 75.00, 300.00, 50.00]
}
transactions_data = {
    'id': [1, 2, 3, 4],
    'date': ['2023-10-01 10:00:00', '2023-10-01 12:30:00', '2023-10-02 15:00:00', '2023-10-03 11:00:00'],
    'location': ['Store A', 'Store B', 'Store A', 'Online'],
    'customers_id': [101, 102, 101, 103]
}
transaction_items_data = {
    'items_id': [201, 202, 203, 202, 204, 205, 201],
    'transactions_id': [1, 1, 2, 2, 3, 3, 4],
    'price': [1200.00, 25.00, 70.00, 25.00, 300.00, 45.00, 1150.00],
    'discount': [100.00, 0.00, 5.00, 0.00, 15.00, 5.00, 50.00]
}

In [ ]:
pd.DataFrame(customers_data).to_sql('customers', source_engine, index=False)
pd.DataFrame(items_data).to_sql('items', source_engine, index=False)
pd.DataFrame(transactions_data).to_sql('transactions', source_engine, index=False)
pd.DataFrame(transaction_items_data).to_sql('transaction_items', source_engine, index=False)

7

In [ ]:
pd.read_sql('SELECT * FROM customers', source_engine)

,id,name,gender
0,101,Alice,P
1,102,Bob,L
2,103,Charlie,L
3,104,David,L


Extract

In [ ]:
try:
    customers = pd.read_sql('SELECT * FROM customers', source_engine)
    items = pd.read_sql('SELECT * FROM items', source_engine)
    transactions = pd.read_sql('SELECT * FROM transactions', source_engine)
    transaction_items = pd.read_sql('SELECT * FROM transaction_items', source_engine)

    transactions['date'] = pd.to_datetime(transactions['date'])

    print("Successfully extracted data from source.")

except Exception as e:
    print(f"An error occurred during extraction: {e}")
    exit()

Successfully extracted data from source.


Transform

In [ ]:
dim_customer = customers.copy()
dim_customer.rename(columns={'id': 'original_id', 'name': 'customer_name'}, inplace=True)
dim_customer['gender'] = dim_customer['gender'].map({'P': 'F', 'L': 'M'})
dim_customer.insert(0, 'customer_key', range(1, 1 + len(dim_customer)))

dim_item = items.copy()
dim_item.rename(columns={'id': 'original_id', 'name': 'item_name', 'price': 'current_price'}, inplace=True)
dim_item.insert(0, 'item_key', range(1, 1 + len(dim_item)))

unique_locations = transactions['location'].unique()
dim_location = pd.DataFrame(unique_locations, columns=['location_name'])
dim_location.insert(0, 'location_key', range(1, 1 + len(dim_location)))
dim_location.sort_values(by='location_key', inplace=True)

unique_dates = transactions['date'].dt.date.unique()
dim_date = pd.DataFrame(unique_dates, columns=['full_date'])
dim_date['full_date'] = pd.to_datetime(dim_date['full_date'])
dim_date.sort_values(by='full_date', inplace=True)
dim_date.insert(0, 'date_key', range(1, 1 + len(dim_date)))
dim_date['year'] = dim_date['full_date'].dt.year
dim_date['quarter'] = dim_date['full_date'].dt.quarter
dim_date['month'] = dim_date['full_date'].dt.month
dim_date['day_of_week'] = dim_date['full_date'].dt.day_name()
dim_date['is_weekend'] = dim_date['full_date'].dt.dayofweek >= 5

In [ ]:
fact_sales = transaction_items.copy()
fact_sales = pd.merge(fact_sales, transactions, left_on='transactions_id', right_on='id', how='left')
fact_sales = pd.merge(fact_sales, dim_customer, left_on='customers_id', right_on='original_id', how='left')
fact_sales = pd.merge(fact_sales, dim_item, left_on='items_id', right_on='original_id', how='left')
fact_sales = pd.merge(fact_sales, dim_location, left_on='location', right_on='location_name', how='left')
fact_sales['date'] = pd.to_datetime(fact_sales['date'].dt.date)
fact_sales = pd.merge(fact_sales, dim_date, left_on='date', right_on='full_date', how='left')
fact_sales['quantity_sold'] = 1
fact_sales['total_amount'] = (fact_sales['price'] * fact_sales['quantity_sold']) - fact_sales['discount']
fact_sales.rename(columns={'transactions_id': 'transaction_id'}, inplace=True)
fact_sales = fact_sales[[
    'date_key', 'customer_key', 'item_key', 'location_key', 'transaction_id',
    'quantity_sold', 'price', 'discount', 'total_amount'
]]

Load

In [ ]:
try:
    dim_customer.to_sql('Dim_Customer', dest_engine, if_exists='replace', index=False)
    dim_item.to_sql('Dim_Item', dest_engine, if_exists='replace', index=False)
    dim_location.to_sql('Dim_Location', dest_engine, if_exists='replace', index=False)
    dim_date.to_sql('Dim_Date', dest_engine, if_exists='replace', index=False)
    fact_sales.to_sql('Fact_Sales', dest_engine, if_exists='replace', index=False)
    print("Successfully loaded all tables to the destination database.")
except Exception as e:
    print(f"\nAn error occurred during loading: {e}")


Successfully loaded all tables to the destination database.


In [ ]:
pd.read_sql('SELECT * FROM Fact_Sales', dest_engine)

,date_key,customer_key,item_key,location_key,transaction_id,quantity_sold,price,discount,total_amount
0,1,1,1,1,1,1,1200.0,100.0,1100.0
1,1,1,2,1,1,1,25.0,0.0,25.0
2,1,2,3,2,2,1,70.0,5.0,65.0
3,1,2,2,2,2,1,25.0,0.0,25.0
4,2,1,4,1,3,1,300.0,15.0,285.0
5,2,1,5,1,3,1,45.0,5.0,40.0
6,3,3,1,3,4,1,1150.0,50.0,1100.0


In [ ]:
try:
    print("--- Dim_Customer ---")
    print(pd.read_sql('SELECT * FROM Dim_Customer', dest_engine))
    print("\n--- Dim_Item ---")
    print(pd.read_sql('SELECT * FROM Dim_Item', dest_engine))
    print("\n--- Dim_Location ---")
    print(pd.read_sql('SELECT * FROM Dim_Location', dest_engine))
    print("\n--- Dim_Date ---")
    print(pd.read_sql('SELECT * FROM Dim_Date', dest_engine))
    print("\n--- Fact_Sales ---")
    print(pd.read_sql('SELECT * FROM Fact_Sales', dest_engine))
except Exception as e:
    print(f"\nAn error occurred during verification: {e}")

--- Dim_Customer ---
   customer_key  original_id customer_name gender
0             1          101         Alice      F
1             2          102           Bob      M
2             3          103       Charlie      M
3             4          104         David      M

--- Dim_Item ---
   item_key  original_id item_name  current_price
0         1          201    Laptop         1200.0
1         2          202     Mouse           25.0
2         3          203  Keyboard           75.0
3         4          204   Monitor          300.0
4         5          205    Webcam           50.0

--- Dim_Location ---
   location_key location_name
0             1       Store A
1             2       Store B
2             3        Online

--- Dim_Date ---
   date_key                   full_date  year  quarter  month day_of_week  \
0         1  2023-10-01 00:00:00.000000  2023        4     10      Sunday   
1         2  2023-10-02 00:00:00.000000  2023        4     10      Monday   
2         3  2023-10

In [ ]:
query = """
    SELECT
        fs.transaction_id,
        dc.customer_name,
        di.item_name,
        dl.location_name,
        dd.full_date,
        fs.total_amount
    FROM Fact_Sales fs
    JOIN Dim_Customer dc ON fs.customer_key = dc.customer_key
    JOIN Dim_Item di ON fs.item_key = di.item_key
    JOIN Dim_Location dl ON fs.location_key = dl.location_key
    JOIN Dim_Date dd ON fs.date_key = dd.date_key
    ORDER BY dd.full_date;
    """
pd.read_sql(query, dest_engine)

,transaction_id,customer_name,item_name,location_name,full_date,total_amount
0,1,Alice,Laptop,Store A,2023-10-01 00:00:00.000000,1100.0
1,1,Alice,Mouse,Store A,2023-10-01 00:00:00.000000,25.0
2,2,Bob,Keyboard,Store B,2023-10-01 00:00:00.000000,65.0
3,2,Bob,Mouse,Store B,2023-10-01 00:00:00.000000,25.0
4,3,Alice,Monitor,Store A,2023-10-02 00:00:00.000000,285.0
5,3,Alice,Webcam,Store A,2023-10-02 00:00:00.000000,40.0
6,4,Charlie,Laptop,Online,2023-10-03 00:00:00.000000,1100.0
